In [1]:
import time
import copy
import numpy as np
import tqdm
import joblib
import os

import pytest

from metadetect.lsst.metadetect import run_metadetect
import descwl_shear_sims as sim


CONFIG = {
    "meas_type": "wmom",
    "metacal": {
        "use_noise_image": True,
        "psf": "fitgauss",
    },
    "psf": {
        "model": "gauss",
        "lm_pars": {},
        "ntry": 2,
    },
    "weight": {
        "fwhm": 1.2,
    },
    "detect": {
        "thresh": 10.0,
    },
}

In [2]:
def _shear_cuts(arr):
    assert arr is not None
    msk = (
        (arr['wmom_flags'] == 0)
        & (arr['wmom_s2n'] > 10)
        & (arr['wmom_T_ratio'] > 1.2)
    )
    return msk


def _meas_shear_data(res):
    msk = _shear_cuts(res['noshear'])
    g1 = np.mean(res['noshear']['wmom_g'][msk, 0])
    g2 = np.mean(res['noshear']['wmom_g'][msk, 1])

    msk = _shear_cuts(res['1p'])
    g1_1p = np.mean(res['1p']['wmom_g'][msk, 0])
    msk = _shear_cuts(res['1m'])
    g1_1m = np.mean(res['1m']['wmom_g'][msk, 0])
    R11 = (g1_1p - g1_1m) / 0.02

    dt = [
        ('g1', 'f8'),
        ('g2', 'f8'),
        ('R11', 'f8'),
    ]
    return np.array([(g1, g2, R11)], dtype=dt)


def _bootstrap_stat(d1, d2, func, seed, nboot=500):
    dim = d1.shape[0]
    rng = np.random.RandomState(seed=seed)
    stats = []
    for _ in tqdm.trange(nboot, leave=False):
        ind = rng.choice(dim, size=dim, replace=True)
        stats.append(func(d1[ind], d2[ind]))
    return stats


def _meas_m_c_cancel(pres, mres):
    x = np.mean(pres['g1'] - mres['g1'])/2
    y = np.mean(pres['R11'] + mres['R11'])/2
    m = x/y/0.02 - 1

    x = np.mean(pres['g2'] + mres['g2'])/2
    y = np.mean(pres['R11'] + mres['R11'])/2
    c = x/y

    return m, c


def _boostrap_m_c(pres, mres):
    m, c = _meas_m_c_cancel(pres, mres)
    bdata = _bootstrap_stat(pres, mres, _meas_m_c_cancel, 14324, nboot=500)
    merr, cerr = np.std(bdata, axis=0)
    return m, merr, c, cerr


In [3]:
!export CATSIM_DIR=/path/to/catsim
from xlens.simulation.simulator.base import SimulateImageHalo
import astropy.table as astable
import fitsio

In [4]:
# Prepare a halo
src_halo = astable.Table()
src_halo["index"] = np.array([1, 2])
src_halo["mass"] = np.array([4e14, 8e14])
src_halo["conc"] = np.array([6.0, 4.0])
src_halo["z_lens"] = np.array([0.2, 0.52])

In [5]:
simulator = SimulateImageHalo("/global/cfs/cdirs/des/zhou/cluster_shear/repos/xlens/examples/cluster/config.ini")

with open("/global/cfs/cdirs/des/zhou/cluster_shear/repos/xlens/examples/cluster/config.ini", "r") as f:
    contents = f.read()
    print(contents)

import os 
os.environ['CATSIM_DIR'] = '/global/cfs/cdirs/des/zhou/cluster_shear/data/catsim'

simulator.run(ifield=1, src_halo=src_halo[1])

[simulation]
root_dir    =   ./
# image directory name
sim_name    =   sim
# catalog directory name
cat_dir     =   test
sum_dir     =   test

# layout
layout = random_disk
# number of rotation
nrot = 2
# number of pixels
coadd_dim = 500
# buff on each side
buff = 20

rotate = False
dither = False

draw_bright = False
star_bleeds = False
cosmic_rays = False
bad_columns = False

psf_variation = 0.0
stellar_density = 0.0
survey_name = LSST

band        =   r
noise_ratio =   1.0
psf_fwhm    =   0.8

Simulating for field: 1, and halo index 2
galsim.Moffat(beta=2.5, scale_radius=0.7076510959648599).transform(1.0,0.0,0.0,1.0)
Simulation has galaxies: 435


In [6]:
sim_data_list = simulator.sim_data_list

In [9]:
sim_data_list[0]['band_data']['i'][0].getPsf()

In [14]:
# def _make_lsst_sim(*, rng, g1, g2, layout):
#     coadd_dim = 400
#     buff = 25

#     galaxy_catalog = sim.galaxies.make_galaxy_catalog(
#         rng=rng,
#         coadd_dim=coadd_dim,
#         buff=buff,
#         layout=layout,
#         gal_type='fixed',
#     )

#     psf = sim.psfs.make_fixed_psf(psf_type='gauss')

#     sim_data = sim.make_sim(
#         rng=rng,
#         galaxy_catalog=galaxy_catalog,
#         coadd_dim=coadd_dim,
#         g1=g1,
#         g2=g2,
#         psf=psf,
#     )
#     return sim_data

# import pickle
# sims_dir = "/global/cfs/cdirs/des/zhou/cluster_shear/data/sims"

def _make_lsst_sim(index, sim_data_list):
    # with open(os.path.join(sims_dir,"sim_data_1_0.pkl"), "rb") as f:_list[0]
    #     res_data = pickle.load(f)_list[0]
    #     sim_data = res_data['sim_data']_list[0]
    #     psf = res_data['psf']_list[0]
    assert sim_data_list[index]['band_data']['i'][0].getPsf() is not None
    return sim_data_list[index]


def _coadd_sim_data(rng, sim_data, nowarp, remove_poisson):
    """
    copied from mdet-lsst-sim
    """
    from descwl_coadd.coadd import make_coadd
    from descwl_coadd.coadd_nowarp import make_coadd_nowarp
    from metadetect.lsst.util import extract_multiband_coadd_data

    bands = list(sim_data['band_data'].keys())
    print("The bands are:", bands)

    if nowarp:
        exps = sim_data['band_data'][bands[0]]

        if len(exps) > 1:
            raise ValueError('only one epoch for nowarp')

        print(exps[0].getPsf())
        
        coadd_data_list = [
            make_coadd_nowarp(
                exp=exps[0],
                psf_dims=sim_data['psf_dims'],
                rng=rng,
                remove_poisson=remove_poisson,
            )
            for band in bands
        ]
    else:
        coadd_data_list = [
            make_coadd(
                exps=sim_data['band_data'][band],
                psf_dims=sim_data['psf_dims'],
                rng=rng,
                coadd_wcs=sim_data['coadd_wcs'],
                coadd_bbox=sim_data['coadd_bbox'],
                remove_poisson=remove_poisson,
            )
            for band in bands
        ]
    return extract_multiband_coadd_data(coadd_data_list)

def _run_sim_one(*, seed, mdet_seed, g1, g2, **kwargs):
    rng = np.random.RandomState(seed=seed)
    # sim_data = _make_lsst_sim(rng=rng, g1=g1, g2=g2, **kwargs)
    sim_data = _make_lsst_sim(index=0, sim_data_list=sim_data_list)

    print(sim_data_list)
    print(sim_data['band_data']['i'][0].getPsf())

    coadd_data = _coadd_sim_data(
        rng=rng, sim_data=sim_data, nowarp=True, remove_poisson=False,
    )

    mdet_rng = np.random.RandomState(seed=mdet_seed)
    results = run_metadetect(
        rng=mdet_rng,
        config=copy.deepcopy(CONFIG),
        **coadd_data,
    )

    return results


def run_sim(seed, mdet_seed, **kwargs):
    # positive shear
    _pres = _run_sim_one(
        seed=seed, mdet_seed=mdet_seed,
        g1=0.02,
        g2=0.0,
        **kwargs,
    )
    if _pres is None:
        return None

    # negative shear
    _mres = _run_sim_one(
        seed=seed, mdet_seed=mdet_seed,
        g1=-0.02,
        g2=0.0,
        **kwargs,
    )
    if _mres is None:
        return None

    return _meas_shear_data(_pres), _meas_shear_data(_mres)


# @pytest.mark.parametrize(
    # 'layout,ntrial', [('grid', 10)]
# )
def test_shear_meas(layout, ntrial):
    nsub = max(ntrial // 100, 10)
    nitr = ntrial // nsub
    rng = np.random.RandomState(seed=116)
    seeds = rng.randint(low=1, high=2**29, size=ntrial)
    mdet_seeds = rng.randint(low=1, high=2**29, size=ntrial)

    tm0 = time.time()

    print("")

    pres = []
    mres = []
    loc = 0
    for itr in tqdm.trange(nitr):
        jobs = [
            joblib.delayed(run_sim)(
                seeds[loc+i], mdet_seeds[loc+i], layout=layout,
            )
            for i in range(nsub)
        ]
        outputs = joblib.Parallel(n_jobs=2, verbose=0, backend='loky')(jobs)

        for out in outputs:
            if out is None:
                continue
            pres.append(out[0])
            mres.append(out[1])
        loc += nsub

        m, merr, c, cerr = _boostrap_m_c(
            np.concatenate(pres),
            np.concatenate(mres),
        )
        print(
            (
                "\n"
                "nsims: %d\n"
                "m [1e-3, 3sigma]: %s +/- %s\n"
                "c [1e-5, 3sigma]: %s +/- %s\n"
                "\n"
            ) % (
                len(pres),
                m/1e-3,
                3*merr/1e-3,
                c/1e-5,
                3*cerr/1e-5,
            ),
            flush=True,
        )

    total_time = time.time()-tm0
    print("time per:", total_time/ntrial, flush=True)

    pres = np.concatenate(pres)
    mres = np.concatenate(mres)
    m, merr, c, cerr = _boostrap_m_c(pres, mres)

    print(
        (
            "\n\nm [1e-3, 3sigma]: %s +/- %s"
            "\nc [1e-5, 3sigma]: %s +/- %s"
        ) % (
            m/1e-3,
            3*merr/1e-3,
            c/1e-5,
            3*cerr/1e-5,
        ),
        flush=True,
    )

    assert np.abs(m) < max(1e-3, 3*merr)
    assert np.abs(c) < 3*cerr
    return


if __name__ == "__main__":
    test_shear_meas(layout="grid", ntrial=10)

  0%|          | 0/1 [00:07<?, ?it/s]Error.  nthreads cannot be larger than environment variable "NUMEXPR_MAX_THREADS" (64)Error.  nthreads cannot be larger than environment variable "NUMEXPR_MAX_THREADS" (64)


AssertionError: 